In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources
from __future__ import print_function
!pip install tensorflow==1.12
import google.datalab.bigquery as bq
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
import datetime

import math
from pandas.io import gbq
import tensorflow as tf
import re
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

In [3]:
query = "SELECT * FROM `sparkline-integrators.tencent_lookalike.export_to_datalab_gender_only` ORDER BY fullVisitorId  LIMIT 100000 " 
data_original = gbq.read_gbq(query,project_id = "sparkline-integrators" )
data= data_original

# remove rows with null values
data.dropna(axis=0, inplace=True)

# remove unused columns
data = data.drop(['visitId','VisitingHour','VisitingDayOfWeek'],axis=1)

# prepare column browser, change browers that are not specified below to 'other'
data.loc[(data.browser != 'Chrome') &
         (data.browser != 'Safari') &
         (data.browser != 'Firefox') &
         (data.browser !='Internet Explorer'),'browser'] = 'other'
# prepare column language
data.rename(columns = {'LANGUAGE':'language'}, inplace=True)
data.loc[data.language.str.contains('en'),'language'] = 'en'  #data has inconsistent values e.g. 'en','en-en','en-bg'
data.loc[data.language.str.contains('th'),'language'] = 'th'
data.language.str.contains('en|th')
data.loc[(data.language!='th') & 
         (data.language!='en'),'language'] = 'other'

# prepare column operatingSystem
data.loc[(data.operatingSystem != 'Windows') &
         (data.operatingSystem != 'Android') &
         (data.operatingSystem != 'Macintosh') &
         (data.operatingSystem !='iOS'),'operatingSystem'] = 'other'

# prepare column source 
data.loc[data.source.str.contains('facebook'),'source'] = 'facebook' #data has inconsistent value such as 'facebook','facebook.com'
data.loc[data.source.str.contains('google'),'source'] = 'google'
data.loc[data.source.str.contains('sanook'),'source'] = 'sanook'
data.loc[data.source.str.contains('direct'),'source'] = 'direct'
data.loc[(data.source != 'facebook') &
         (data.source != 'google') &
         (data.source != 'sanook') &
         (data.source !='direct'),'source'] = 'other'

# remove rows where contentCategory contains non-english characters
data = data[data.contentCategory.str.contains('^[A-Za-z]+$', regex=True)]
data.loc[:,'contentCategory'] = data.contentCategory.str.lower()
data.loc[:,'userGender'] = data.userGender.str.lower()
# female=0, male=1
data.userGender.replace({
  'female':'0',
  'male':'1'
}, inplace=True)  


print(data.shape)

def make_pivot_table(columns, _id):
    result =None
    for column in columns:
        pivot = data.loc[:,[_id,column]]
        pivot = pivot.pivot_table(index=id,columns=column,aggfunc= any)
        pivot.columns = ['_'.join((column,i)) for i  in pivot.columns]
        if result is None:
          result = pivot
        else:
          result = result.join(pivot)
        
    result = result.replace({
      True:1,
      None:0
    })
    
    return result

gender = data.loc[:,['fullVisitorId','userGender']].drop_duplicates().set_index('fullVisitorId')    
data = make_pivot_table(data.columns[1:-1],'fullVisitorId')
data = data.join(gender)

#remove columns with less than 0.1% of 1s  
data = data[data.columns[data.sum()>data.shape[0]*0.001]]

Requesting query... ok.
Job ID: 30c12983-b150-4f0b-ad09-f9f6eb7a3894
Query running...
Query done.
Cache hit.

Retrieving results...
Got 100000 rows.

Total time taken 18.43 s.
Finished at 2019-01-16 05:44:50.
(56075, 8)


In [4]:
train_dataset, test_dataset, train_labels, test_labels = train_test_split(data.drop('userGender',axis=1), data['userGender'], test_size = 0.2, random_state = 100)


In [9]:
print('Training Features Shape:', train_dataset.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_dataset.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (12033, 75)
Training Labels Shape: (12033,)
Testing Features Shape: (3009, 75)
Testing Labels Shape: (3009,)


In [96]:
rf = RandomForestClassifier(n_estimators = 1000, verbose = 1)
rf.fit(train_dataset, train_labels)
predictions = rf.predict(test_dataset)
accuracy= (predictions == test_labels).sum()/test_labels.shape[0]
print("accuracy rate = {}".format(accuracy))

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   20.3s finished


accuracy rate = 0.5596543702226653


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    1.0s finished
